# Pneumonia-Treatment-RAG

In [2]:
!pip install PyPDF2
!pip install transformers torch
!pip install sentence_transformers
!pip install sentencepiece
!pip install langchain_openai
!pip install openai 
!pip install qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.9 MB/s eta 0:00:00:00:01


# **SentenceTransformer Embedding **

In [4]:
from sentence_transformers import SentenceTransformer
def get_sentenceTF_embeddings(sentences):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  embeddings =[]
  for chunk in sentences:
    embeddings.append(model.encode(chunk))
  print(len(embeddings))
  return embeddings

def Embed_stenteceTF(sentence):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  return model.encode(sentence)

# **Sementic Chuncking**

In [5]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans


def get_bert_embeddings(sentences):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    # Tokenize sentences
    inputs = tokenizer(sentences, return_tensors='pt', padding=True, truncation=True, max_length=128)

    # Get BERT embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Use the [CLS] token representation as the sentence embedding
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()

    return embeddings

def semantic_chunking_with_attention(text, n_clusters=5, max_chunk_size=512):
    # Split the input text into sentences based on '.'
    sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]

    # Get BERT embeddings for sentences
    embeddings = get_bert_embeddings(sentences)

    # Perform KMeans clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=5).fit(embeddings)
    labels = kmeans.labels_

    # Group sentences by cluster
    clusters = {}
    for sentence, label in zip(sentences, labels):
        if label not in clusters:
            clusters[label] = []
        clusters[label].append(sentence)

    # Create chunks from clusters with max_chunk_size constraint
    chunks = []
    for cluster in clusters.values():
        current_chunk = ""
        for sentence in cluster:
            if len(current_chunk) + len(sentence) + 1 <= max_chunk_size:  # +1 for the period
                current_chunk += sentence + ". "
            else:
                chunks.append(current_chunk.strip())
                current_chunk = sentence + ". "
        if current_chunk:
            chunks.append(current_chunk.strip())

    return chunks

# **Recursive Chunking**

In [6]:
def division_Chunk(text, chunk_size=400):
    sentences = text.split(".")
    chunk1 = ""
    chunks = []

    for sentence in sentences:
        word_count = len(sentence.split())

        if word_count > chunk_size:
            chunks.extend(Recursive_Chunking(sentence, chunk_size))
        else:
            if len(chunk1.split()) + word_count <= chunk_size:
                chunk1 += sentence + "\n"
            else:
                chunks.append(chunk1.strip())
                chunk1 = sentence + "\n"

    if chunk1.strip():
        chunks.append(chunk1.strip())

    return chunks

def Recursive_Chunking(text, chunk_size=1000):
    chunks = text.split("\n\n")
    #print(len(chunks))
    result_chunks = []

    for chunk in chunks:
        word_count = len(chunk.split())

        if word_count > chunk_size:
            middle = int(0.8 * word_count)
            words = chunk.split()
            part1 = " ".join(words[:middle])
            part2 = " ".join(words[middle:])

            result_chunks.extend(division_Chunk(part1, chunk_size))
            result_chunks.extend(division_Chunk(part2, chunk_size))
        else:
            result_chunks.append(chunk.strip())

    return result_chunks

# Add_Data_From_PDF_file

In [7]:
from PyPDF2 import PdfReader
import re

def read_pdf_file(file_path):
    points = []
    # Open the PDF file
    with open(file_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        # Iterate through each page in the PDF
        for page in pdf_reader.pages:
            text = page.extract_text()
            if text:
                points.append(text.strip())
    return points

def replace_double_spaces(text):
    # Replace all occurrences of two consecutive spaces with '\xa0'
    cleaned_text = re.sub(r'\xa0',' ' , text)
    cleaned_text1 = re.sub(r'\u200b','' , cleaned_text)
    return cleaned_text1

def read_pdf_file_(file_path, Chunking_type, Max_Chunk_size, NumberOf_cluster=20):
    Pneumonia = read_pdf_file(file_path)
    for i in range(len(Pneumonia)):
        Pneumonia[i] = replace_double_spaces(Pneumonia[i])
    Pneumonia_Str = "\n".join(Pneumonia)
    if Chunking_type == "Recursive":
        chunks = Recursive_Chunking(Pneumonia_Str, chunk_size=Max_Chunk_size)
    elif Chunking_type == "Semantic":
        chunks = semantic_chunking_with_attention(Pneumonia_Str, n_clusters=NumberOf_cluster, max_chunk_size=Max_Chunk_size)
    embedding = get_sentenceTF_embeddings(chunks)
    return embedding, chunks

# **PrepareData**

In [8]:
# Read Data from pdf file
embeddings, chunks = read_pdf_file_(file_path='/kaggle/input/pneumoniadocuments/(Public Health in the 21st Century) Micaela L. Surez_ Steffani M. Ortega - Pneumonia_ Symptoms Diagnosis and Treatment _ Symptoms Diagnosis and Treatment-Nova Science Publishers Incorporated (2011.pdf',Chunking_type='Semantic',Max_Chunk_size = 600,NumberOf_cluster = 20)
print(len(chunks))
print(len(embeddings))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

964
964
964


In [52]:
from qdrant_client.models import PointStruct
import uuid

# Create PointStruct objects
points = [
    PointStruct(
        id=str(uuid.uuid4()),  # Use a unique ID for each document
        vector=embedding,  # Ensure this is a list of floats
        payload={"text": doc}  # Optional metadata
    )
    for i, (doc, embedding) in enumerate(zip(chunks, embeddings))
]

In [53]:
from qdrant_client import QdrantClient
import json

with open('config.json', 'r') as file:
    config = json.load(file)


url = config['qdrant_url']
api_key = config['qdrant_api_key']

# Initialize Qdrant client
qdrant_client = QdrantClient(url=url, api_key=api_key)

In [54]:
from qdrant_client.models import CollectionParams, VectorParams

# Define the dimension of your vectors and distance metric
vector_dim = 384  # Dimension of your vectors
distance_metric = "Cosine"  # Distance metric (use "Cosine" as string)

# Define vector configuration
vector_params = VectorParams(size=vector_dim, distance=distance_metric)

# Create the collection
qdrant_client.create_collection(
    collection_name="pneumoniaBot5",
    vectors_config=vector_params  # Correct argument name
)

True

In [55]:
# Insert points into Qdrant
qdrant_client.upsert(
    collection_name="pneumoniaBot5",
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [57]:
query = ["How to treat pneumonia"]
query_embedding_response = get_sentenceTF_embeddings(query)
# print(query_embedding_response)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

1


In [59]:
# Assuming query_embedding_response is a list of lists or a NumPy array
query_embedding = query_embedding_response[0].tolist()  # Convert to list if necessary

In [60]:
# Perform similarity search
results = qdrant_client.search(
    collection_name="pneumoniaBot5",
    query_vector=query_embedding,

)
# Print results
for result in results:
    print(result)

id='c74d69a3-af68-40f0-bc35-6c5389fc00e8' version=0 score=0.68566144 payload={'text': 'Despite \navailability of effective antibiotic treatment the mortality remains 6 to 12% in patient s \nadmitted to hospital with community acquired pneumonia and over 50% in patients admitted \nto the intensive care unit (figures from the United Kingdom). Supportive treatment explored in patients with pneumonia includes oxygen in a \nvariety of applications, fluid therapy regimens, chest physiotherapy, steroid treatment, \ngranulocyte colony stimulating factor treatment, surfactant application and vitamins A, C and \nD, zinc, protein and calorie supplements.'} vector=None shard_key=None order_value=None
id='c49b98a7-8033-48a0-b863-a58ce9b28118' version=0 score=0.6829466 payload={'text': 'The \nobjective of this review is to summarize evidence  for the effectiveness of modes of \nsupportive treatment in patients with pneumonia. It is the largest killer, accounting  \nfor 28 to 34% of all child deaths 

In [41]:
# NVIDIA NIM API
from openai import OpenAI
import json

with open('config.json', 'r') as file:
    config = json.load(file)

client = OpenAI(
  base_url = config['nvidia_base_url'],
  api_key = config['nvidia_api_key']
)

In [61]:
def custom_prompt(query: str, results):
    # Extract the page content from the results
    source_knowledge = "\n".join([x.payload['text'] for x in results])
    
    # Create the augmented prompt
    augment_prompt = f"""Using the contexts below, answer the query:

    Contexts:
    {source_knowledge}

    Query: {query}"""
    
    return augment_prompt

In [63]:
# print(custom_prompt(query, results))

In [64]:
messages = []
# messages.append({"role":"assistant", "content": full_response})
prompt = {"role":"system", "content": custom_prompt(query, results)}
messages.append(prompt)

res = client.chat.completions.create(
  model="meta/llama-3.1-8b-instruct",
  messages=messages,
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
  stream=True
)

full_response = ""
for chunk in res:
  if chunk.choices[0].delta.content is not None:
#     print(chunk.choices[0].delta.content, end="")
    full_response += chunk.choices[0].delta.content
print(full_response)

The query can be answered based on the provided contexts. 

Supportive treatment for pneumonia includes:

1. Oxygen therapy in various applications
2. Fluid therapy regimens
3. Chest physiotherapy
4. Steroid treatment
5. Granulocyte colony stimulating factor treatment
6. Surfactant application
7. Vitamins A, C, and D
8. Zinc supplements
9. Protein and calorie supplements

In addition to supportive treatment, the following diagnostic procedures can be used:

1. Computed Tomography (CT) or other useful procedures for the management of pneumonia
2. Transtracheal aspiration
3. Thoracentesis
4. Closed pleural biopsy
5. Bronchoscopy with aspiration and brushing
6. Transbronchial biopsy
7. Percutaneous aspiration
8. Open pulmonary or pleural biopsies

Prompt and appropriate antibiotic therapy is also crucial in improving the prognosis of pneumonia, especially in immunocompromised patients.
